In [1]:
pip install feedparser

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# --- Standard Library ---
import os
import requests

# --- Third-Party Libraries ---
import pandas as pd
from bs4 import BeautifulSoup
import feedparser
from dotenv import load_dotenv

# --- LangChain / AI Related ---

from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd

from langchain.embeddings.base import Embeddings
from typing import List

In [17]:
type(results)

list

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Teste e definição de ferramentas de web scrapping (ou com propósito similar)

Aqui foram testadas algumas ferramentas de web scraping, inicialmente tentando extrair notícias da fonte https://busca.ebc.com.br/?site_id=agenciabrasil&q=SRAG (Agência Brasil), que parece conter uma boa quantidade de informações relacionadas à SRAG's.


### Teste web scrapping

In [5]:
def scrape_agencia_brasil(query: str, max_pages: int = 1):
    """
    Scrape news articles from Agência Brasil search results.

    Args:
        query (str): Search keyword (e.g., "SRAG").
        max_pages (int): Number of pages of results to scrape.

    Returns:
        pd.DataFrame: Table with columns [title, link, summary, date].
    """
    base_url = "https://busca.ebc.com.br/"
    results = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}?site_id=agenciabrasil&q={query}&page={page}"
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()

        soup = BeautifulSoup(resp.text, "html.parser")

        # Cada resultado está dentro de <li> na lista
        for item in soup.select("ul.search-results > li"):
            title_el = item.select_one("h2 a")
            summary_el = item.select_one("p")
            date_el = item.select_one("span.date")

            title = title_el.get_text(strip=True) if title_el else None
            link = title_el["href"] if title_el else None
            summary = summary_el.get_text(strip=True) if summary_el else None
            date = date_el.get_text(strip=True) if date_el else None

            results.append({
                "title": title,
                "link": link,
                "summary": summary,
                "date": date
            })

    return pd.DataFrame(results)


In [6]:
df_news = scrape_agencia_brasil("SRAG", max_pages=1)
print(df_news.head())


Empty DataFrame
Columns: []
Index: []


DataFrame vazio. O site da Agência Brasil utiliza JavaScript para carregar o conteúdo dinamicamente, então métodos simples de scraping com requests e BeautifulSoup não conseguem capturar os títulos das notícias (pelo menos dentro do meu conhecimento).

### Teste RSS parser (mais eficiente e mais interessante, pode fazer mais sentido em um escopo de POC)

In [7]:
# URL do feed RSS
rss_url = "https://agenciabrasil.ebc.com.br/rss/saude/feed.xml"

# Faz o parse do feed
feed = feedparser.parse(rss_url)

# Filtra notícias que contenham "SRAG" no título ou descrição
resultados = []
for entry in feed.entries:
    titulo = entry.title
    descricao = entry.get("summary", "")
    link = entry.link

    print(titulo)


Infecções por Aedes aegypti elevam risco de complicações no parto
Sociedade de Reumatologia defende mais centros de infusão no SUS
Saúde lança campanha para reduzir recusa à doação de órgãos
Capacitação de generalistas pode agilizar jornada do paciente, diz SBR
Internação por Influenza A e Covid cresce no Distrito Federal e Goiás
Câmara aprova MP que cria programa Agora Tem Especialistas
Anvisa: não há registros que relacionem paracetamol a autismo
Uma em cada 4 pessoas já pensou em suicídio, alerta pesquisa
Anvisa proíbe venda de café torrado, whey e suplementos; veja marcas
Ministério da Saúde reafirma que paracetamol não causa autismo


Infelizmente não contempla todas as notícias, apenas as principais que não incluem nada sobre SRAG.

Necessário tentar algum outro método.

### Teste utilizando a API do site newsapi 

Site de busca que contém api que busca informações de diversas fontes e disponibiliza título, descrição resumida e data das notícias mais relevantes por palavra-chave (no nosso caso, pela palavra SRAG)

In [8]:
load_dotenv()

API_KEY =  os.getenv("NEWSAPI_KEY")
QUERY = "SRAG"
LANGUAGE = "pt"
PAGE_SIZE = 20  # How many news are displayed

url = "https://newsapi.org/v2/everything"

params = {
    "q": QUERY,
    "language": LANGUAGE,
    "pageSize": PAGE_SIZE,
    "sortBy": "publishedAt",
    "apiKey": API_KEY
}

response = requests.get(url, params=params)
data = response.json()

if data.get("status") == "ok":
    articles = data.get("articles", [])
    if articles:
        # Transform news to a dataframe 
        df_news = pd.DataFrame([
            {
                "title": art["title"],
                "description": art["description"],
                "url": art["url"],
                "publishedAt": art["publishedAt"],
                "source": art["source"]["name"]
            }
            for art in articles
        ])
else:
    print("Erro na requisição:", data.get("message"))


In [9]:
df_news["title"] = df_news["title"].fillna("no-title")

In [10]:
df_news

,title,description,url,publishedAt,source
0,"Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista","A disseminação da gripe aviária H5N1 no continente americano, em especial nos EUA, aumenta a probabilidade do vírus se adaptar aos humanos \nThe post Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista appeared first on InfoMo…",https://oglobo.globo.com/saude/noticia/2025/09/21/o-risco-de-uma-nova-pandemia-nesse-momento-vem-principalmente-das-americas-afirma-virologista.ghtml,2025-09-21T15:28:18Z,Globo
1,no-title,Oito estados e cinco capitais estão em nível de alerta ou risco para SRAG nas últimas duas semanas,https://www1.folha.uol.com.br/equilibrioesaude/2025/09/infeccoes-respiratorias-graves-apresentam-tendencia-de-aumento-no-brasil-aponta-boletim-da-fiocruz.shtml,2025-09-18T21:37:11Z,Uol.com.br
2,Casos de doenças respiratórias aumentam e DF entra em nível de alerta,Boletim da Fiocruz revela crescimento de ocorrências recentes e indicativo de tendência de alta; queda na vacinação pode ser um dos fatores,https://www.metropoles.com/distrito-federal/casos-de-doencas-respiratorias-aumentam-e-df-entra-em-nivel-de-alerta,2025-09-14T17:17:40Z,Metropoles.com
3,83% das mortes por gripe em Campinas em 2025 são de pessoas sem vacina,"Campinas soma 49 óbitos por Síndrome Respiratória Aguda Grace, que é provocada pelo vírus Influenza. \nDivulgação\nA Prefeitura de Campinas confirmou mais três mortes por Síndrome Respiratória Aguda Grave (SRAG), que é provocada pelo vírus Influenza. Ao todo, a…",https://g1.globo.com/sp/campinas-regiao/noticia/2025/08/26/83percent-das-mortes-por-gripe-em-campinas-em-2025-sao-de-pessoas-sem-vacina.ghtml,2025-08-26T21:17:49Z,Globo


## Criação da vectorstore para a busca

Para a criação da base de dados vetorial e embeddings serão utilizadas duas ferramentas principais:

- FAISS:
FAISS é um banco vetorial rápido e eficiente, ideal para armazenar e buscar grandes volumes de vetores. Ele permite que o agente encontre rapidamente notícias relevantes, mesmo em datasets grandes, garantindo consultas por similaridade em tempo real.

- OpenAI Embeddings:
Os embeddings da OpenAI transformam textos em vetores que capturam o significado das palavras e sentenças. Isso possibilita buscas semânticas no banco vetorial, recuperando notícias relacionadas a SRAG mesmo que a redação seja diferente, dando contexto preciso para o relatório.


In [11]:
df_news

,title,description,url,publishedAt,source
0,"Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista","A disseminação da gripe aviária H5N1 no continente americano, em especial nos EUA, aumenta a probabilidade do vírus se adaptar aos humanos \nThe post Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista appeared first on InfoMo…",https://oglobo.globo.com/saude/noticia/2025/09/21/o-risco-de-uma-nova-pandemia-nesse-momento-vem-principalmente-das-americas-afirma-virologista.ghtml,2025-09-21T15:28:18Z,Globo
1,no-title,Oito estados e cinco capitais estão em nível de alerta ou risco para SRAG nas últimas duas semanas,https://www1.folha.uol.com.br/equilibrioesaude/2025/09/infeccoes-respiratorias-graves-apresentam-tendencia-de-aumento-no-brasil-aponta-boletim-da-fiocruz.shtml,2025-09-18T21:37:11Z,Uol.com.br
2,Casos de doenças respiratórias aumentam e DF entra em nível de alerta,Boletim da Fiocruz revela crescimento de ocorrências recentes e indicativo de tendência de alta; queda na vacinação pode ser um dos fatores,https://www.metropoles.com/distrito-federal/casos-de-doencas-respiratorias-aumentam-e-df-entra-em-nivel-de-alerta,2025-09-14T17:17:40Z,Metropoles.com
3,83% das mortes por gripe em Campinas em 2025 são de pessoas sem vacina,"Campinas soma 49 óbitos por Síndrome Respiratória Aguda Grace, que é provocada pelo vírus Influenza. \nDivulgação\nA Prefeitura de Campinas confirmou mais três mortes por Síndrome Respiratória Aguda Grave (SRAG), que é provocada pelo vírus Influenza. Ao todo, a…",https://g1.globo.com/sp/campinas-regiao/noticia/2025/08/26/83percent-das-mortes-por-gripe-em-campinas-em-2025-sao-de-pessoas-sem-vacina.ghtml,2025-08-26T21:17:49Z,Globo


Criando uma classe  que funciona como um adaptador que permite utilizar modelos do `SentenceTransformer` dentro do LangChain. Ela implementa os métodos `embed_documents` e `embed_query` exigidos pela interface `Embeddings`, retornando vetores numéricos que podem ser indexados em um VectorStore (como FAISS).  

O objetivo é padronizar o uso de embeddings, permitindo buscas por similaridade de forma consistente, sem alterar a API do LangChain.


In [12]:
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t) for t in texts]

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text)

Realizando um primeiro teste utilizando o título e a descrição da notícia como conteúdos para a base vetorial 

In [13]:
# Creating list of documents
documents = []
for _, row in df_news.iterrows():
    content = f"{row['title']}. {row['description']}"
    metadata = {
        "title": row['title'],
        "url": row['url'],
        "publishedAt": row['publishedAt'],
        "source": row['source']
    }
    documents.append(Document(page_content=content, metadata=metadata))

# Creating embeddings wrapper
embedding_wrapper = SentenceTransformerEmbeddings()

# Creating 
#  index from documents
vectorstore = FAISS.from_documents(documents, embedding_wrapper)

# Exemplo de busca
query = "SRAG aumento de casos/taxa de vacinação/taxa de mortalidade/taxa de internação em UTI/ Brasil"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print("Título:", doc.metadata["title"])
    print("URL:", doc.metadata["url"])
    print("Publicado em:", doc.metadata["publishedAt"])
    print("Fonte:", doc.metadata["source"])
    print("Resumo:", doc.page_content, "...\n")

Título: 83% das mortes por gripe em Campinas em 2025 são de pessoas sem vacina
URL: https://g1.globo.com/sp/campinas-regiao/noticia/2025/08/26/83percent-das-mortes-por-gripe-em-campinas-em-2025-sao-de-pessoas-sem-vacina.ghtml
Publicado em: 2025-08-26T21:17:49Z
Fonte: Globo
Resumo: 83% das mortes por gripe em Campinas em 2025 são de pessoas sem vacina. Campinas soma 49 óbitos por Síndrome Respiratória Aguda Grace, que é provocada pelo vírus Influenza. 
Divulgação
A Prefeitura de Campinas confirmou mais três mortes por Síndrome Respiratória Aguda Grave (SRAG), que é provocada pelo vírus Influenza. Ao todo, a… ...

Título: Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista
URL: https://oglobo.globo.com/saude/noticia/2025/09/21/o-risco-de-uma-nova-pandemia-nesse-momento-vem-principalmente-das-americas-afirma-virologista.ghtml
Publicado em: 2025-09-21T15:28:18Z
Fonte: Globo
Resumo: Risco de nova pandemia nesse momento vem principalmente das Américas, diz v

In [14]:
print(df_news[df_news["title"] == "Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista"]["description"].item())


A disseminação da gripe aviária H5N1 no continente americano, em especial nos EUA, aumenta a probabilidade do vírus se adaptar aos humanos 
The post Risco de nova pandemia nesse momento vem principalmente das Américas, diz virologista appeared first on InfoMo…


A notícia de Campinas pode ser relevante, mas a outra não faz muito sentido pois envolve os EUA. 

Vamos criar a base vetorial com base no título e url apenas, para ver se melhores resultados podem ser obtidos. 

In [54]:
# ---------Creating list of documents ---------
documents = []
for _, row in df_news.iterrows():
    content = f"{row['title']} {row['url']}"
    metadata = {
        "title": row['title'],
        "url": row['url'],
        "publishedAt": row['publishedAt'],
        "source": row['source'],
        "description": row['description']
    }
    documents.append(Document(page_content=content, metadata=metadata))

# --------- Creating a wrapper class for SentenceTransformer --------
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self.model.encode(t) for t in texts]

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text)

# --------- Creating embeddings wrapper ---------
embedding_wrapper = SentenceTransformerEmbeddings()

# ---------Creating FAISS index from documents  ---------
vectorstore = FAISS.from_documents(documents, embedding_wrapper)

# --------- Search example ---------
query = "SRAG aumento de casos/taxa de vacinação/taxa de mortalidade/taxa de internação em UTI/ Brasil"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print("Título:", doc.metadata["title"])
    print("URL:", doc.metadata["url"])
    print("Publicado em:", doc.metadata["publishedAt"])
    print("Fonte:", doc.metadata["source"])
    print("Description", doc.metadata["description"])

Título: 83% das mortes por gripe em Campinas em 2025 são de pessoas sem vacina
URL: https://g1.globo.com/sp/campinas-regiao/noticia/2025/08/26/83percent-das-mortes-por-gripe-em-campinas-em-2025-sao-de-pessoas-sem-vacina.ghtml
Publicado em: 2025-08-26T21:17:49Z
Fonte: Globo
Description Campinas soma 49 óbitos por Síndrome Respiratória Aguda Grace, que é provocada pelo vírus Influenza. 
Divulgação
A Prefeitura de Campinas confirmou mais três mortes por Síndrome Respiratória Aguda Grave (SRAG), que é provocada pelo vírus Influenza. Ao todo, a…
Título: no-title
URL: https://www1.folha.uol.com.br/equilibrioesaude/2025/09/infeccoes-respiratorias-graves-apresentam-tendencia-de-aumento-no-brasil-aponta-boletim-da-fiocruz.shtml
Publicado em: 2025-09-18T21:37:11Z
Fonte: Uol.com.br
Description Oito estados e cinco capitais estão em nível de alerta ou risco para SRAG nas últimas duas semanas


In [15]:
vectorstore

Trouxe notícias mais relevante perante o tema/métricas que foram calculadas.

Por enquanto será escolhida essa abordagem de banco de dados vetorial/ busca por similaridade

Trabalhos futuros podem trazer mais robustez tanto ao processo de RAG quanto à forma de se extrair as notícias (a API utilizada possui limite de caracteres, o que pode trazer problemas de interpretação para o modelo)